### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes five files: 

1. A file with one record for each airport, 
2. A file with one record for each airline, 
3. A file with one record for each each route, 
4. A file with one record for each type of airplane, and
5. A file with one record for each country.  

Each file has a relationship to the others, with some fields in common.  To make best use of the data, we will need to combine all three files.  This is the core challenge of relational databases.  This lesson looks at how to do that, specifically using the merge() function in python.  

Credits:

This lesson draws from the Data Analysis and Visualization in Python for Ecologists course by Data Carpentry:

https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html

and from the pandas documentation:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


### Learning Objectives: 

1. Understand the meaning of relational data and be able to identify common keys. 

2. Understand the different types of relational joins: concatenate, inner join, outer join, left join, right join. 

3. Use the pandas syntax pd.concat() and pd.merge() to perform those relational joins. 

4. Start to build an awareness of open data resources, and of documentation/tutorials for ongoing learning. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["ID", "NAME", "CITY", "COUNTRY", "IATA", "ICAO", "LATITUDE", "LONGITUDE", "ALTITUDE", "TIMEZONE", "DST", "TZ", "TYPE", "SOURCE"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["ID", "NAME", "ALIAS", "IATA", "ICAO", "CALLSIGN", "COUNTRY", "ACTIVE"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["AIRLINE", "AIRLINE_ID", "SOURCE", "SOURCE_ID", "DEST", "DEST_ID", "CODESHARE", "STOPS", "EQUIPMENT"]

# Read in the planes data.
planes = pd.read_csv("data/planes.dat", header=None, na_values='\\N')
planes.columns = ["NAME", "IATA", "ICAO"]

# Read in the countries data.
countries = pd.read_csv("data/countries.dat", header=None, na_values='\\N')
countries.columns = ["NAME", "ISO", "DAFIF"]

### Start by seeing what's in the data.  

What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

Start with the airports table

In [ ]:
# printing the first few rows gives us a good sense of what is in there
airports.head()

In [ ]:
# we can also look a the data types
# alternatively, we could summarize the data as we did in the previous lesson
airports.dtypes

##### Your turn:  

Spend a few minutes examining the remaining tables.  See if you can identify which fields are common across multiple tables.  These are the "keys".  Together, we will make sure we have identified the right keys. 

In [ ]:
airlines.columns

In [ ]:
routes.columns

In [ ]:
planes.columns

In [ ]:
countries.columns

In [ ]:
airports.head()

In [ ]:
airlines.head()


In [ ]:
routes.head()

In [ ]:
routes[routes.AIRLINE == '2B'].head(3)

In [ ]:
countries.head()

In [ ]:
planes[planes.IATA == 'CR2']

### Selecting data

We might want to work with a subset of data.  If so, we can select data just like we did in the previous lesson. 

In [ ]:
# we can select all airports in one of the least visited countries on earth
airports[airports['COUNTRY']=="Tuvalu"]

In [ ]:
# we can select all routes to that airport
routes[routes['DEST']=='FUN']

In [ ]:
# and routes from that airport
routes[routes['SOURCE']=='FUN']

In [ ]:
# where are these flights going to/from? 


In [ ]:
airports[airports.IATA == 'ORD']

#####  Your turn: 

Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [ ]:
lex_routes = routes[(routes.DEST == 'LEX') | (routes.SOURCE == 'LEX')]
lex_routes

In [ ]:
len(lex_routes[lex_routes.DEST == 'LEX'])

### Concatentating data

To concatenate is to link things together in a series or chain.  It is similar to, but more powerful than appending data, because it can work on either rows or columns, is faster, and will perform set logic (union or intersection).  

In [ ]:
# let's go back and contatenate the routes both to and from Tuvalu
to_tuvalu = routes[routes['DEST']=='FUN']
from_tuvalu = routes[routes['SOURCE']=='FUN']


In [ ]:
# the basic function, given a list of dataframes to concatenate
# note that we call concatenate as a "static" method, rather than a method 
# associated with a specific object. 

pd.concat([to_tuvalu, from_tuvalu])

In [ ]:
# if we want to use the dataframe later, we need to assign it to a variable name

tuvalu_routes = pd.concat([to_tuvalu, from_tuvalu])

Note to see more about this method, we can examine the pandas docs:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [ ]:
# note that one of the arguments is axis.  
# axis=0 indicates that the rows should be concatenated, 
# while axis=1 indicates to concatenate the columns 

pd.concat([to_tuvalu, from_tuvalu], axis=1)

##### Your turn:

Can you use the concat() function to create one dataframe with flights both into and out of LEX?

### Merging data

Sometimes we want to merge, or join, the data instead of concatenating it.  For example, maybe we hate flying on small planes, so want to know what kind of aircraft we would need to fly on to visit Tuvalu.  This data comes from a different table. 

In [ ]:
# we see that the Tuvalu routes operate with AT5 equipment
tuvalu_routes

In [ ]:
# but what is an AT5? Let's look in the planes table. 
planes.head()

In [ ]:
# in this case, we are lucky, and it is in the first few rows.  But to do this systemactically, we
# would want to join the tables.  To do this, we use the merge() function in pandas.  

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA')

Let's stop and look at what is going on in the above command. 

It is important to understand the type of merge, as specified by the how option.  Here is what the pandas docs say:


![alt text](img/how.png)


![alt text](img/how2.png)

So it looks like the above merge defaulted to an inner join


![alt text](img/inner.png)

In [ ]:
# we can check this by explicitly setting the option and confirming that we get the same result  
# next, let's check what happens if we choose the other options

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='inner')

In [ ]:
# outer join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='outer')

In [ ]:
# left join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='left')

In [ ]:
# right join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='right')

I find that I most often want a left join, as described here: 

![alt text](img/left.png)

It may be worth examining some of the other options used in merge().  We can refer to the pandas documentation to learn more. 

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

In [ ]:
lex_routes.reset_index()

routes, airlines, planes, airports, countries 

#### Your turn

Now let's look at which airlines operate in and out of Lexington.  How many routes does each airline have?  

First, let's spend some time planning together.  Then individually take some time to implement the code we planned. 

In [ ]:
lex_air = pd.merge(lex_routes, airlines, left_on= 'AIRLINE_ID', right_on= 'ID', how = 'inner')
lex_air.head()
lex_air['international'] = None
for x in lex_air.index:
    lex_air['international'][x] = 'True' if ((lex_air['NAME'][x] == 'Air France') | (lex_air['NAME'][x] == 'KLM Royal Dutch Airlines')) else 'false'

In [ ]:
#which airlines operate inand out of lexington?
lex_air

In [ ]:
airlist = ['Pinnacle Airlines', 'American Airlines', 'Air France','Delta Air Lines', 'Allegiant Air', 'KLM Royal Dutch Airlines','United Airlines', 'US Airways']
nroute = [len(lex_air[lex_air['NAME'] == x ]) for x in airlist]
nroutedict = {'NumberofRoutes' : nroute}


In [ ]:
a = {'airline': ['Pinnacle Airlines', 'American Airlines', 'Air France',
       'Delta Air Lines', 'Allegiant Air', 'KLM Royal Dutch Airlines',
       'United Airlines', 'US Airways']}
dd = pd.DataFrame(a)
#pd.concat(dd, pd.DataFrame(nroutedict))
nr =pd.DataFrame(nroutedict)
dd['Number of routes'] = nr.NumberofRoutes[0:]

In [ ]:
dd

In [ ]:
#How many airlines operate out of Lexington?
len(lex_air.NAME.unique())

#### It looks like there are some international airlines with Lexington routes.  

To look at how many routes they have, create a new column in your dataframe called 'International', which is set to True for an overseas airline and False for a domestic airline.  Calculate the percent of routes with an overseas airline.  

You should have the tools you need to answer this question using what we learned in last week's lesson.  Again, let's start by planning together, then implement the code on your own.  

In [ ]:
percentinternational = (len(lex_air[lex_air.international == 'True']) / len(lex_air)) * 100

In [ ]:
percentinternational

#### Codeshares

It looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

It is this sort of messiness that will consume much of your effort in research and data science.  It is very important to "become one with your data". 

Figuring this out will be the first part of your homework.  

### Review Questions: 

1. What is a key in relational data? 

2. Explain the difference between a left join and an outer join.  

3. Explain the difference between pd.concat() and pd.merge()

4. Identify two possible places to look for help if you get stuck on the homework. 

### Homework 1 - Code shares

Dig into the data as described above, and do your best to identify which routes are code shares, and how many unique routes operate into/out of Lexington.

In [ ]:
lex_routes = routes[(routes.DEST == 'LEX') | (routes.SOURCE == 'LEX')].reset_index()
lex_routes.tail()

In [ ]:
lex_duplicate_routes = lex_routes[lex_routes['CODESHARE'].notnull()].reset_index()

In [ ]:
lex_unique_routes = lex_routes[lex_routes['CODESHARE'].isnull()].reset_index()
lex_unique_routes.tail()

In [ ]:
a = []
for x in lex_unique_routes.index:
    a.append((lex_unique_routes.SOURCE[x],lex_unique_routes.DEST[x]))
b = []
for x in lex_unique_routes.index:
    b.append((lex_unique_routes.DEST[x],lex_unique_routes.SOURCE[x]))

In [ ]:
lex_unique_routes['OD'] = a[0:]
lex_unique_routes['DO'] = b[0:]


In [ ]:
lex_unique_routes.drop_duplicates('OD', inplace = True)

In [ ]:
lex_unique_routes

In [ ]:
lex_unique_routes.OD[0] not in c

In [ ]:
duplicate_routes = routes[routes['CODESHARE'].notnull()].reset_index()

In [ ]:
unique_routes = routes[routes['CODESHARE'].isnull()].reset_index()

In [ ]:
lex_unique_routes.OD[0] == lex_unique_routes.DO[1]

In [ ]:
lex_duplicate_routes

### Homework 2 - The Kevin Bacon Game

Of course, it is possible to fly beyond those few cities from Lexington--you just have to transfer to get there.  Your task is to:

1. Identify which airports you can get to from Lexington with only one transfer.  
2. Identify which airports you can get to from Lexington with two transfers.

You have the tools to do this using the merge functionality presented today, but you will need to be smart about what you are merging.  

Next class, we will use these data to make interactive maps of these flight routes, similar to what you see in the back of the airline magazines.  


#### Bonus: 

This is the air travel version of the Kevin Bacon game (https://oracleofbacon.org/). What is the number N, such that you can reach every airport in the world with N or fewer transfers?

#### Extra Bonus: 

Use this very important piece of knowledge to impress your friends at parties!

In [3]:
routes.head()

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2


In [4]:
airports.head()

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [94]:
d1 = routes[routes['SOURCE'] == 'LEX'].copy()
d1.drop_duplicates(subset = 'DEST', keep = 'first', inplace = True)
d1 = d1[['DEST']].copy()
d1

,DEST
3588,ATL
5763,CLT
5764,DFW
5765,ORD
21096,DCA
21097,DTW
21098,LGA
21099,MSP
29047,FLL
29048,PGD


In [95]:
d2 = d1.merge(right = routes, how = 'inner', left_on = 'DEST', right_on = 'SOURCE')

In [96]:
d2.drop_duplicates('DEST_y', keep = 'first')

,DEST_x,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST_y,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,ATL,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
1,ATL,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
2,ATL,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
3,ATL,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
4,ATL,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
5,ATL,3M,20710.0,ATL,3682.0,TUP,5773.0,NaN,0,SF3
6,ATL,9E,3976.0,ATL,3682.0,AZO,4039.0,NaN,0,CRJ
7,ATL,9E,3976.0,ATL,3682.0,CHA,3578.0,NaN,0,CRJ
8,ATL,9E,3976.0,ATL,3682.0,CID,4043.0,NaN,0,CRJ
9,ATL,9E,3976.0,ATL,3682.0,CRW,4285.0,NaN,0,CRJ


In [97]:
d3 = d2.set_index('DEST_y', drop = False).copy()
len(d3)

3558

### Number of airports on 1 transfer

In [109]:
d4 = d3.drop(index = ['DFW', 'CLT','DFW', 'ORD', 'DCA','DTW','LGA','MSP','FLL','IAH']).copy()
d4 = d4.drop_duplicates('DEST_y', keep = 'first')
d4 = d4[['AIRLINE', 'SOURCE', 'DEST_y', 'EQUIPMENT']]
#d3.drop(index = 'CLT')
#['ATL','CLT','DFW','ORD','DCA','DTW','LGA','MSP','FLL','PGD','PIE','SFB','IAH']
len(d4.index.unique())
d5 = d4.copy()

### Two Transfers

In [116]:
d5 = d5.reset_index(drop =True)
d5.head()

,AIRLINE,SOURCE,DEST_y,EQUIPMENT
0,3M,ATL,LWB,SF3
1,3M,ATL,MCN,SF3
2,3M,ATL,MEI,SF3
3,3M,ATL,MSL,SF3
4,3M,ATL,PIB,SF3


In [141]:
d6 = d5.merge(right = routes, how = 'inner', left_on = 'DEST_y', right_on = 'SOURCE')

In [142]:
d6.head()

,AIRLINE_x,SOURCE_x,DEST_y,EQUIPMENT_x,AIRLINE_y,AIRLINE_ID,SOURCE_y,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT_y
0,3M,ATL,LWB,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3
1,3M,ATL,LWB,SF3,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0,SF3
2,3M,ATL,MCN,SF3,3M,20710.0,MCN,3754.0,ATL,3682.0,NaN,0,SF3
3,3M,ATL,MCN,SF3,3M,20710.0,MCN,3754.0,MCO,3878.0,NaN,0,SF3
4,3M,ATL,MEI,SF3,3M,20710.0,MEI,4335.0,ATL,3682.0,NaN,0,SF3


In [144]:
d7 = d6[['DEST']].copy()
d7 = d7.drop_duplicates('DEST', keep = 'first')
d7 = d7.set_index('DEST', drop = False)
len(d7)

1803

In [145]:
d7 = d7.drop(index = ['DFW', 'CLT','DFW', 'ORD', 'DCA','DTW','LGA','MSP','FLL','IAH','PGD','PIE','SFB'])
len(d7)

1791

In [159]:
d7 = d7.drop(index = list(d6.DEST_y), errors = 'ignore')
d7.reset_index(drop = True)
len(d7)

1455

In [160]:
d7

,DEST
DEST,
GLH,GLH
AZA,AZA
MMU,MMU
CGN,CGN
HAM,HAM
BOM,BOM
ATH,ATH
ABZ,ABZ
ACC,ACC


In [158]:
d7[d7.DEST == 'GLH']

,DEST
DEST,
GLH,GLH


make a set. it won't contain duplicates
pd.apply()
lambda functions.